# Analysis of Enriched Pharmacy Data

## Notebook Configurations

In [156]:
import pandas as pd
import plotly.express as px
import datetime as dt

In [157]:
pharmacies = pd.read_csv('pharmacy_data.csv')
pharmacies

,Pharmacy Name,License Number,License Type,License Status,Expiration Date,City,State,County,Zip,LAT,LONG,isGovernment,isSatellite,Facility Type
0,986 INFUSION PHARMACY #1 INC,LSC 101414,Sterile Compounding Pharmacy,CLEAR,2025-07-01,ANAHEIM,CA,ORANGE,92807,33.848733,-117.788357,False,False,Infusion Center
1,A & O SPECIALTY PHARMACY,LSC 101817,Sterile Compounding Pharmacy,CLEAR,2025-03-01,SALINAS,CA,MONTEREY,93901,36.636344,-121.623978,False,False,NaN
2,ACCREDO HEALTH GROUP INC,NSC 101280,Nonresident Sterile Compounding Pharmacy,CLEAR,2025-12-01,WARRENDALE,PA,NaN,15086,40.674947,-80.106364,False,False,Infusion Center
3,ACCREDO HEALTH GROUP INC,NSC 101279,Nonresident Sterile Compounding Pharmacy,CLEAR,2025-12-01,ORLANDO,FL,NaN,32822,28.489898,-81.290154,False,False,Infusion Center
4,ADVANCE CARE PHARMACY,LSC 99611,Sterile Compounding Pharmacy,CLEAR,2025-07-01,ESCONDIDO,CA,SAN DIEGO,92025,33.102005,-117.080419,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836,WEST ANAHEIM MEDICAL CENTER,LSC 101464,Sterile Compounding Pharmacy,CLEAR,2025-11-01,ANAHEIM,CA,ORANGE,92804,33.818246,-117.974985,False,False,Medical Facility
837,WHITE MEMORIAL MEDICAL CENTER DBA ADVENTIST HE...,LSC 100146,Sterile Compounding Pharmacy,CLEAR,2025-07-01,LOS ANGELES,CA,LOS ANGELES,90033,34.050380,-118.211991,False,False,Medical Facility
838,WHITTIER HOSPITAL MEDICAL CENTER,LSC 101408,Sterile Compounding Pharmacy,CLEAR,2026-01-01,WHITTIER,CA,LOS ANGELES,90605,33.950295,-118.024352,False,False,Medical Facility
839,"WOMENS INTERNATIONAL PHARMACY, INC. DBA BELMAR...",NSC 101754,Nonresident Sterile Compounding Pharmacy,CLEAR,2025-03-01,YOUNGTOWN,AZ,NaN,85363,33.584969,-112.304916,False,False,NaN


In [158]:
pharmacies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 841 entries, 0 to 840
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Pharmacy Name    841 non-null    object 
 1   License Number   841 non-null    object 
 2   License Type     841 non-null    object 
 3   License Status   841 non-null    object 
 4   Expiration Date  841 non-null    object 
 5   City             841 non-null    object 
 6   State            841 non-null    object 
 7   County           792 non-null    object 
 8   Zip              841 non-null    int64  
 9   LAT              828 non-null    float64
 10  LONG             828 non-null    float64
 11  isGovernment     841 non-null    bool   
 12  isSatellite      841 non-null    bool   
 13  Facility Type    773 non-null    object 
dtypes: bool(2), float64(2), int64(1), object(9)
memory usage: 80.6+ KB


## What Type of Facility is Being Licensed? 

### Facility Type

In [159]:
# Calculate the count of each facility type
facility_counts = pharmacies['Facility Type'].value_counts().reset_index()
facility_counts.columns = ['Facility Type', 'Count']

# Create the funnel chart
funnel_fig = px.funnel(facility_counts, x='Count', y='Facility Type', 
                       title="Distribution of Facility Types",
                       labels={'Count': 'Number of Facilities', 'Facility Type': 'Type of Facility'})
funnel_fig.show()

In [160]:
donut_fig = px.pie(facility_counts, names='Facility Type', values='Count', title='Distribution of Facility Types', hole = 0.25, labels = {'Facility Type': 'Type of Facility', 'Count' : 'Number of Facilities'}, template='simple_white')
donut_fig.show()

In [161]:
bar_fig = px.bar(
    facility_counts,
    x = 'Count',
    y = 'Facility Type',
    orientation = 'h',
    title = 'What Types of Facilities Hold Sterile Compounding Licenses in California?',
    labels = {'Facility Type': 'Type of Facility', 'Count' : 'Number of Facilities'},
    template = 'seaborn'
)

bar_fig.show()

### License Type

In [162]:
# Calculate the count of each facility type
license_counts = pharmacies['License Type'].value_counts().reset_index()
license_counts.columns = ['License Type', 'Count']

In [163]:
donut_fig = px.pie(license_counts, names='License Type', values='Count', title='Distribution of License Types', hole = 0.25, labels = {'License Type': 'Type of License', 'Count' : 'Number of Facilities'}, template='simple_white')
donut_fig.show()

## Geography: Exploring Where Licensed Pharmacies are Located

In [164]:
# Drop rows with null values in LAT and LONG
pharmacy_geo = pharmacies.dropna(subset=['LAT', 'LONG'])

# Create the map
map_fig = px.scatter_mapbox(pharmacy_geo, 
                            lat='LAT', 
                            lon='LONG', 
                            hover_name='Pharmacy Name',  
                            color='Facility Type',  
                            title="Where are the licensed pharmacies located? ",
                            zoom=6,  # Default zoom level
                            center={"lat": 36.7783, "lon": -119.4179},  # Center on California
                            height=600,  
                            template='simple_white')  

# Add mapbox token
map_fig.update_layout(mapbox_style="carto-positron") 

# Allow user interaction and zoom
map_fig.update_layout(mapbox=dict(zoom=6, center={"lat": 36.7783, "lon": -119.4179}))

# Show the map
map_fig.show()

In [165]:
pharmacy_geo = pharmacies.dropna(subset=['City', 'County'])
pharmacy_cities = pharmacy_geo.groupby(['City', 'Facility Type']).size().reset_index(name='Count').sort_values(by='Count')
pharmacy_counties = pharmacy_geo.groupby(['County', 'Facility Type']).size().reset_index(name='Count').sort_values(by='Count')

In [166]:
pharmacy_cities

,City,Facility Type,Count
303,WESTMINSTER,Medical Facility,1
302,WEST HILLS,Medical Facility,1
0,ALAMEDA,Infusion Center,1
300,WATSONVILLE,Medical Facility,1
313,YUCCA VALLEY,Medical Facility,1
...,...,...,...
218,SACRAMENTO,Medical Facility,13
84,FRESNO,Medical Facility,15
225,SAN DIEGO,Medical Facility,21
229,SAN FRANCISCO,Medical Facility,25


In [167]:
# Ensure the DataFrame is sorted by County
pharmacy_counties_sorted = pharmacy_counties.sort_values(by='County')

# Create the bubble chart
bubble_fig = px.scatter(pharmacy_counties_sorted, 
                        x='Facility Type', 
                        y='County', 
                        size='Count', 
                        color='Facility Type',
                        title="Facilities by County (Bubble Chart)",
                        labels={'Count': 'Number of Facilities', 'County': 'County'},
                        template='plotly_white',
                        category_orders={"County": pharmacy_counties_sorted['County'].unique()})  # Enforce alphabetical order

bubble_fig.show()

In [168]:
# Ensure the DataFrame is sorted by City
pharmacy_cities_sorted = pharmacy_cities.sort_values(by='City')

# Create the bubble chart for cities
city_bubble_fig = px.scatter(pharmacy_cities_sorted, 
                             x='Facility Type', 
                             y='City', 
                             size='Count', 
                             color='Facility Type',
                             title="Facilities by City (Bubble Chart)",
                             labels={'Count': 'Number of Facilities', 'City': 'City'},
                             template='plotly_white',
                             category_orders={"City": pharmacy_cities_sorted['City'].unique()})  # Enforce alphabetical order

city_bubble_fig.show()

## Exploring Expiration Dates

In [169]:
# Ensure the Expiration Date is in the correct datetime format
pharmacies['Expiration Date'] = pd.to_datetime(pharmacies['Expiration Date'], errors='coerce')

# Filter out rows with invalid Expiration Date
pharmacies_expiration = pharmacies[pharmacies['Expiration Date'].notna()]

# Extract year and month
pharmacies_expiration['Year'] = pharmacies_expiration['Expiration Date'].dt.year
pharmacies_expiration['Month'] = pharmacies_expiration['Expiration Date'].dt.month

# Filter out rows where the Year is 2024
pharmacies_expiration = pharmacies_expiration[pharmacies_expiration['Year'] != 2024]

# Group by Year and Month and count the number of expirations
expirations_by_year_month = pharmacies_expiration.groupby(['Year', 'Month']).size().reset_index(name='Count')

# Filter out rows where the count of expirations is zero
expirations_by_year_month = expirations_by_year_month[expirations_by_year_month['Count'] > 0]

# Get unique years present in the data
available_years = expirations_by_year_month['Year'].unique()

# Create the heatmap of expirations by year and month
heatmap_fig = px.density_heatmap(expirations_by_year_month, 
                                 x='Month', 
                                 y='Year', 
                                 z='Count',  # Use 'Count' for the intensity
                                 title="Pharmacy Expirations Heatmap",
                                 labels={'Month': 'Month', 'Year': 'Year', 'Count': 'Number of Expirations'},
                                 nbinsx=12, nbinsy=5, 
                                 color_continuous_scale='Viridis')

# Ensure the Y-axis is treated as categorical and only shows available years
heatmap_fig.update_layout(
    yaxis=dict(
        tickmode='array',
        tickvals=available_years,  # Show only available years
        type='category'  # Make sure it's treated as a categorical axis
    )
)

heatmap_fig.show()


In [170]:
pharmacies_expiration

,Pharmacy Name,License Number,License Type,License Status,Expiration Date,City,State,County,Zip,LAT,LONG,isGovernment,isSatellite,Facility Type,Year,Month
0,986 INFUSION PHARMACY #1 INC,LSC 101414,Sterile Compounding Pharmacy,CLEAR,2025-07-01,ANAHEIM,CA,ORANGE,92807,33.848733,-117.788357,False,False,Infusion Center,2025,7
1,A & O SPECIALTY PHARMACY,LSC 101817,Sterile Compounding Pharmacy,CLEAR,2025-03-01,SALINAS,CA,MONTEREY,93901,36.636344,-121.623978,False,False,NaN,2025,3
2,ACCREDO HEALTH GROUP INC,NSC 101280,Nonresident Sterile Compounding Pharmacy,CLEAR,2025-12-01,WARRENDALE,PA,NaN,15086,40.674947,-80.106364,False,False,Infusion Center,2025,12
3,ACCREDO HEALTH GROUP INC,NSC 101279,Nonresident Sterile Compounding Pharmacy,CLEAR,2025-12-01,ORLANDO,FL,NaN,32822,28.489898,-81.290154,False,False,Infusion Center,2025,12
4,ADVANCE CARE PHARMACY,LSC 99611,Sterile Compounding Pharmacy,CLEAR,2025-07-01,ESCONDIDO,CA,SAN DIEGO,92025,33.102005,-117.080419,False,False,NaN,2025,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836,WEST ANAHEIM MEDICAL CENTER,LSC 101464,Sterile Compounding Pharmacy,CLEAR,2025-11-01,ANAHEIM,CA,ORANGE,92804,33.818246,-117.974985,False,False,Medical Facility,2025,11
837,WHITE MEMORIAL MEDICAL CENTER DBA ADVENTIST HE...,LSC 100146,Sterile Compounding Pharmacy,CLEAR,2025-07-01,LOS ANGELES,CA,LOS ANGELES,90033,34.050380,-118.211991,False,False,Medical Facility,2025,7
838,WHITTIER HOSPITAL MEDICAL CENTER,LSC 101408,Sterile Compounding Pharmacy,CLEAR,2026-01-01,WHITTIER,CA,LOS ANGELES,90605,33.950295,-118.024352,False,False,Medical Facility,2026,1
839,"WOMENS INTERNATIONAL PHARMACY, INC. DBA BELMAR...",NSC 101754,Nonresident Sterile Compounding Pharmacy,CLEAR,2025-03-01,YOUNGTOWN,AZ,NaN,85363,33.584969,-112.304916,False,False,NaN,2025,3


In [171]:
# Ensure the Expiration Date is in the correct datetime format
pharmacies['Expiration Date'] = pd.to_datetime(pharmacies['Expiration Date'], errors='coerce')

# Filter out rows with invalid Expiration Date
pharmacies_expiration = pharmacies[pharmacies['Expiration Date'].notna()]

# Extract year and month
pharmacies_expiration['Year'] = pharmacies_expiration['Expiration Date'].dt.year
pharmacies_expiration['Month'] = pharmacies_expiration['Expiration Date'].dt.month

# Filter out rows where the Year is 2024
pharmacies_expiration = pharmacies_expiration[pharmacies_expiration['Year'] != 2024]

# Group by Year and Month and count the number of expirations
expirations_by_year_month = pharmacies_expiration.groupby(['Year', 'Month']).size().reset_index(name='Count')

# Filter out rows where the count of expirations is zero
expirations_by_year_month = expirations_by_year_month[expirations_by_year_month['Count'] > 0]

# Create a "Date" column combining Year and Month for the line plot
expirations_by_year_month['Date'] = pd.to_datetime(expirations_by_year_month[['Year', 'Month']].assign(DAY=1))

# Create the line chart of expirations over time
line_fig = px.line(expirations_by_year_month, 
                   x='Date', 
                   y='Count', 
                   title="Pharmacy Expirations Over Time",
                   labels={'Count': 'Number of Expirations', 'Date': 'Date'},
                   markers=True,  # Mark data points on the line
                   template='plotly_white')

line_fig.show()


In [172]:
# Ensure the Expiration Date is in the correct datetime format
pharmacies['Expiration Date'] = pd.to_datetime(pharmacies['Expiration Date'], errors='coerce')

# Filter out rows with invalid Expiration Date
pharmacies_expiration = pharmacies[pharmacies['Expiration Date'].notna()]

# Extract year, month, and facility type
pharmacies_expiration['Year'] = pharmacies_expiration['Expiration Date'].dt.year
pharmacies_expiration['Month'] = pharmacies_expiration['Expiration Date'].dt.month

# Filter out rows where the Year is 2024
pharmacies_expiration = pharmacies_expiration[pharmacies_expiration['Year'] != 2024]

# Group by Year, Month, and Facility Type and count the number of expirations
expirations_by_year_month_type = pharmacies_expiration.groupby(['Year', 'Month', 'Facility Type']).size().reset_index(name='Count')

# Filter out rows where the count of expirations is zero
expirations_by_year_month_type = expirations_by_year_month_type[expirations_by_year_month_type['Count'] > 0]

# Create a "Date" column combining Year and Month for the area chart
expirations_by_year_month_type['Date'] = pd.to_datetime(expirations_by_year_month_type[['Year', 'Month']].assign(DAY=1))

# Sort by 'Count' within each 'Date' so the smallest counts appear at the bottom
expirations_by_year_month_type = expirations_by_year_month_type.sort_values(by=['Date', 'Count'], ascending=[True, True])

# Create the stacked area chart of expirations over time by facility type
area_fig = px.area(expirations_by_year_month_type, 
                   x='Date', 
                   y='Count', 
                   color='Facility Type', 
                   title="Pharmacy Expirations Over Time by Facility Type",
                   labels={'Count': 'Number of Expirations', 'Date': 'Date'},
                   template='plotly_white',
                   markers=True)  # Add markers to show data points

# Show the plot
area_fig.show()


In [173]:
import plotly.graph_objects as go

# Ensure the Expiration Date is in the correct datetime format
pharmacies['Expiration Date'] = pd.to_datetime(pharmacies['Expiration Date'], errors='coerce')

# Filter out rows with invalid Expiration Date
pharmacies_expiration = pharmacies[pharmacies['Expiration Date'].notna()]

# Extract year, month, and facility type
pharmacies_expiration['Year'] = pharmacies_expiration['Expiration Date'].dt.year
pharmacies_expiration['Month'] = pharmacies_expiration['Expiration Date'].dt.month

# Filter out rows where the Year is 2024
pharmacies_expiration = pharmacies_expiration[pharmacies_expiration['Year'] != 2024]

# Group by Year, Month, and Facility Type and count the number of expirations
expirations_by_year_month_type = pharmacies_expiration.groupby(['Year', 'Month', 'Facility Type']).size().reset_index(name='Count')

# Filter out rows where the count of expirations is zero
expirations_by_year_month_type = expirations_by_year_month_type[expirations_by_year_month_type['Count'] > 0]

# Create a "Date" column combining Year and Month for the plot
expirations_by_year_month_type['Date'] = pd.to_datetime(expirations_by_year_month_type[['Year', 'Month']].assign(DAY=1))

# Calculate total expirations per month (sum across all facility types)
total_expirations_by_month = expirations_by_year_month_type.groupby('Date')['Count'].sum().reset_index(name='Total Expirations')

# Create a bar chart for expirations by Facility Type and Month
bar_fig = go.Figure()

# Add bars for each Facility Type
for facility_type in expirations_by_year_month_type['Facility Type'].unique():
    df_facility = expirations_by_year_month_type[expirations_by_year_month_type['Facility Type'] == facility_type]
    bar_fig.add_trace(go.Bar(x=df_facility['Date'], 
                             y=df_facility['Count'], 
                             name=facility_type, 
                             marker=dict(color=px.colors.qualitative.Set1[len(bar_fig.data) % len(px.colors.qualitative.Set1)])))

# Add a line for the total expirations
bar_fig.add_trace(go.Scatter(x=total_expirations_by_month['Date'], 
                             y=total_expirations_by_month['Total Expirations'], 
                             mode='lines+markers', 
                             name='Total Expirations', 
                             line=dict(color='black', width=3)))

# Update layout for better visualization
bar_fig.update_layout(
    title="Pharmacy Expirations by Facility Type with Total Expirations",
    xaxis_title="Date",
    yaxis_title="Number of Expirations",
    template="plotly_white",
    barmode="stack",  # Stack bars for each month by facility type
    xaxis=dict(tickformat="%b %Y"),
    height=600
)

bar_fig.show()


In [174]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

# Ensure the Expiration Date is in the correct datetime format
pharmacies['Expiration Date'] = pd.to_datetime(pharmacies['Expiration Date'], errors='coerce')

# Filter out rows with invalid Expiration Date
pharmacies_expiration = pharmacies[pharmacies['Expiration Date'].notna()]

# Extract year, month, and facility type
pharmacies_expiration['Year'] = pharmacies_expiration['Expiration Date'].dt.year
pharmacies_expiration['Month'] = pharmacies_expiration['Expiration Date'].dt.month

# Filter out rows where the Year is 2024
pharmacies_expiration = pharmacies_expiration[pharmacies_expiration['Year'] != 2024]

# Group by Year, Month, and Facility Type and count the number of expirations
expirations_by_year_month_type = pharmacies_expiration.groupby(['Year', 'Month', 'Facility Type']).size().reset_index(name='Count')

# Filter out rows where the count of expirations is zero
expirations_by_year_month_type = expirations_by_year_month_type[expirations_by_year_month_type['Count'] > 0]

# Create a "Date" column combining Year and Month for the plot
expirations_by_year_month_type['Date'] = pd.to_datetime(expirations_by_year_month_type[['Year', 'Month']].assign(DAY=1))

# Calculate total expirations per month (sum across all facility types)
total_expirations_by_month = expirations_by_year_month_type.groupby('Date')['Count'].sum().reset_index(name='Total Expirations')

# Create a bar chart for expirations by Facility Type and Month
bar_fig = go.Figure()

# Add bars for each Facility Type
for facility_type in expirations_by_year_month_type['Facility Type'].unique():
    df_facility = expirations_by_year_month_type[expirations_by_year_month_type['Facility Type'] == facility_type]
    bar_fig.add_trace(go.Bar(x=df_facility['Date'], 
                             y=df_facility['Count'], 
                             name=facility_type, 
                             marker=dict(color=px.colors.qualitative.Set1[len(bar_fig.data) % len(px.colors.qualitative.Set1)])))

# Add a line for the total expirations
bar_fig.add_trace(go.Scatter(x=total_expirations_by_month['Date'], 
                             y=total_expirations_by_month['Total Expirations'], 
                             mode='lines+markers', 
                             name='Total Expirations', 
                             line=dict(color='black', width=3)))

# Update layout for better visualization
bar_fig.update_layout(
    title="Pharmacy Expirations by Facility Type with Total Expirations",
    xaxis_title="Date",
    yaxis_title="Number of Expirations",
    template="plotly_white",
    barmode="group",  # Group bars side by side
    xaxis=dict(tickformat="%b %Y"),
    height=600
)

bar_fig.show()


## Specialties 

In [175]:
# Create pharmacies_specialties where Specialties is not null
pharmacies_specialties = pharmacies.copy()
pharmacies_specialties = pharmacies_specialties[pharmacies_specialties['Specialties'].notna()]

# Ensure the Specialties column is a list (if not already)
pharmacies_specialties['Specialties'] = pharmacies_specialties['Specialties'].apply(lambda x: x.split(',') if isinstance(x, str) else x)


KeyError: 'Specialties'

In [ ]:
# Assuming your dataframe is named pharmacies
# Exploding the list of specialties to individual rows
specialties = pharmacies.explode('Specialties')

# Counting the occurrences of each specialty
specialty_counts = specialties['Specialties'].value_counts().reset_index()
specialty_counts.columns = ['Specialty', 'Count']

# Creating the bar plot
fig = px.bar(specialty_counts, x='Specialty', y='Count', title="Specialty Frequency", 
             labels={'Specialty': 'Pharmacy Specialties', 'Count': 'Number of Pharmacies'})
fig.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming your dataframe is named pharmacies

# Step 1: Explode the 'Specialties' column so that each pharmacy-specialty pair becomes a separate row
exploded_specialties = pharmacies.explode('Specialties')

# Step 2: Create a one-hot encoded DataFrame of specialties per pharmacy
specialty_matrix = pd.get_dummies(exploded_specialties['Specialties'])

# Step 3: Aggregate the data by summing the one-hot encoded values
# This creates a matrix where each row is a pharmacy and each column is a specialty
specialty_matrix = specialty_matrix.groupby(exploded_specialties.index).sum()

# Step 4: Calculate co-occurrence matrix by multiplying the specialty matrix with its transpose
co_occurrence_matrix = specialty_matrix.T.dot(specialty_matrix)

# Step 5: Plot the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(co_occurrence_matrix, cmap='Blues', annot=True, fmt='d', square=True)
plt.title('Specialty Co-occurrence Heatmap')
plt.show()


In [ ]:
# Drop rows with NA values in the 'Specialties' column
pharmacies_clean = pharmacies.copy()
pharmacies_clean = pharmacies_clean.dropna(subset='Specialties')
pharmacies_clean

In [ ]:
# Split each string by commas and strip any extra whitespace
exploded_specialties = pharmacies_clean['Specialties'].str.split(',').explode()

# Remove leading/trailing spaces from each specialty
exploded_specialties = exploded_specialties.str.strip()

# Get unique specialties
unique_specialties = exploded_specialties.unique()

# Display the unique specialties
unique_specialties


In [ ]:
import re

In [190]:
# Drop rows where 'Specialties' column is either NaN or an empty string
pharmacies_clean = pharmacies_clean[pharmacies_clean['Specialties'].notna()]
pharmacies_clean = pharmacies_clean[pharmacies_clean['Specialties'] != '']

# Alternatively, you can combine both conditions into one filter:
# pharmacies_clean = pharmacies_clean[~pharmacies_clean['Specialties'].isin([None, ''])]

# Now continue with your code
pharmacies_clean['Specialties'] = pharmacies_clean['Specialties'].str.replace('Ear, Nose, and Throat', 'Ear Nose and Throat', regex=False)

# Fix camel case, split combined specialties like 'PainManagementGastroenterology' into 'Pain Management' and 'Gastroenterology'
pharmacies_clean['Specialties'] = pharmacies_clean['Specialties'].apply(
    lambda x: re.sub(r'([a-z])([A-Z])', r'\1 \2', x) if isinstance(x, str) else x
)


# Split and process the specialties as before
exploded_specialties = pharmacies_clean['Specialties'].str.split(',').explode()

# Remove leading/trailing spaces from each specialty
exploded_specialties = exploded_specialties.str.strip()

# Count how many times each specialty appears (i.e., how many pharmacies have that specialty)
specialty_counts = exploded_specialties.value_counts()

# Display the unique specialties with their counts
specialty_counts


Specialties
Hormone Replacement Therapy (HRT)    16
Veterinary                           11
Dermatology                          11
Pain Management                      11
Weight Loss                          11
Oncology                             10
IV Vitamin Therapy                    9
Ophthalmology                         8
Total Parenteral Nutrition (TPN)      6
Pediatrics                            5
Nuclear Medicine                      5
Molecular Imaging                     5
Dentistry                             5
Gastroenterology                      4
Anti-Aging                            4
Clinical Trials                       4
Antibiotics                           3
Hematology                            3
Podiatry                              3
Nuclear Cardiology                    3
Sexual Health                         3
Anesthesia                            2
Sports Medicine                       2
Ear Nose and Throat                   2
Long Term Care              

In [191]:
specialty_counts

Specialties
Hormone Replacement Therapy (HRT)    16
Veterinary                           11
Dermatology                          11
Pain Management                      11
Weight Loss                          11
Oncology                             10
IV Vitamin Therapy                    9
Ophthalmology                         8
Total Parenteral Nutrition (TPN)      6
Pediatrics                            5
Nuclear Medicine                      5
Molecular Imaging                     5
Dentistry                             5
Gastroenterology                      4
Anti-Aging                            4
Clinical Trials                       4
Antibiotics                           3
Hematology                            3
Podiatry                              3
Nuclear Cardiology                    3
Sexual Health                         3
Anesthesia                            2
Sports Medicine                       2
Ear Nose and Throat                   2
Long Term Care              

In [192]:
import plotly.express as px

# Ensure specialty counts are sorted by count in descending order
specialty_counts_sorted = specialty_counts.sort_values(ascending=False)

# Plot for Specialty Counts (sorted)
specialty_chart = px.bar(
    x=specialty_counts_sorted.values,
    y=specialty_counts_sorted.index,
    orientation='h',
    labels={'x': 'Count', 'y': 'Specialty'},
    title="Specialty Counts",
    color=specialty_counts_sorted.values,  # Adding color based on the count for more clarity
    color_continuous_scale='Viridis'  # A visually appealing color scale
)

# Reversing the y-axis to have the largest values on top
specialty_chart.update_layout(
    template="simple_white",  # Simple white background
    yaxis={'categoryorder': 'total ascending'},  # Sorting values in descending order
    xaxis_title="Count",
    yaxis_title="Specialty",
    margin={"l": 150, "r": 200},  # Increase both left and right margin for better label space
    height=1200,  # Increase height to give more room for the labels
    yaxis_tickangle=0  # Horizontal labels
)

# Ensure condition counts are sorted by count in descending order
condition_counts_sorted = condition_counts.sort_values(ascending=False)


# Plot for Condition Counts (sorted)
condition_chart = px.bar(
    x=condition_counts_sorted.values,
    y=condition_counts_sorted.index,
    orientation='h',
    labels={'x': 'Count', 'y': 'Condition'},
    title="Condition Counts",
    color=condition_counts_sorted.values,  # Adding color based on the count for more clarity
    color_continuous_scale='Viridis'  # A visually appealing color scale
)

# Reversing the y-axis to have the largest values on top
condition_chart.update_layout(
    template="simple_white",  # Simple white background
    yaxis={'categoryorder': 'total ascending'},  # Sorting values in descending order
    xaxis_title="Count",
    yaxis_title="Condition",
    margin={"l": 150, "r": 200},  # Increase both left and right margin for better label space
    height=1200,  # Increase height to give more room for the labels
    width=1200,  # Increase width for more space for labels
    yaxis_tickangle=0,  # Horizontal labels
    font=dict(size=10),  # Decrease font size for labels
)
# Show the charts
specialty_chart.show()
condition_chart.show()
